In [ ]:
import os
import glob
import re

from collections import namedtuple

import time
from datetime import datetime, timedelta

import matplotlib.pyplot as plt

import numpy as np

import itertools

import time

import cv2

In [ ]:
from krwordrank.word import KRWordRank

from wordcloud import WordCloud

import matplotlib.pyplot as plt

In [ ]:
log_file_list = glob.glob('../chats/*.log')

In [ ]:
print(log_file_list)

In [ ]:
log_file = log_file_list[-2]

In [ ]:
Chat = namedtuple('Chat', 'time id text')

In [ ]:
t0 = datetime.strptime('00:00:00','%H:%M:%S')

In [ ]:
chat_history = list()

with open(log_file, 'r') as file:
    
    count = 0
    for chat in file:
#     chat = next(file).strip()
#         print(chat.strip())
        chat_time = re.search('\[.*?\]', chat)[0]
        user_id_match = re.search('<.*?>', chat)
        user_id = user_id_match[0]
        start_of_chat = user_id_match.span()[-1]
        text = chat[start_of_chat:].strip()
        
        dt = datetime.strptime(chat_time,'[%H:%M:%S]') - t0
#         print(dt.total_seconds())
        
        chat_tuple = Chat(dt.total_seconds(), user_id[1:-1], text)

#         print('{}'.format(chat_tuple))
        chat_history.append(chat_tuple)
        

In [ ]:
chat_time_list = [getattr(chat, 'time') for chat in chat_history]

In [ ]:
max_min = np.int(np.ceil(max(chat_time_list)/60) + 1)

In [ ]:
def express_bins_as_time_range(bins, index_list):
    
    time_range_list = list()
    
    for index in index_list:
        begin = timedelta(seconds=int(bins[index]))
        end = timedelta(seconds=int(bins[index+1]))
        print('[{} ~ {}]'.format(begin, end))
        time_range_list.append((begin, end))
        
    return time_range_list
    
def express_bins_as_sec_range(bins, index_list):
    
    time_range_list = list()
    
    for index in index_list:
        begin = int(bins[index])
        end = int(bins[index+1])
        time_range_list.append((begin, end))
        
    return time_range_list

In [ ]:
plt.figure(figsize=(18, 3))
histogram, bins, _ = plt.hist(chat_time_list, bins=max_min)
plt.show()

plt.figure(figsize=(18, 6))
plt.plot(bins[:-1], histogram)
historgram_avg = np.convolve(histogram, np.ones(5)/5, 'same')
plt.plot(bins[:-1], historgram_avg)
plt.show()

histogram_delta = histogram - historgram_avg

print(np.mean(histogram_delta))

print(np.std(histogram_delta))

outlier_idx = np.argwhere(histogram_delta > np.mean(histogram_delta) + 2.5 * np.std(histogram_delta))
outlier_idx = list(itertools.chain.from_iterable(outlier_idx))

print(outlier_idx)
express_bins_as_time_range(bins, outlier_idx)

plt.figure(figsize=(18, 6))
plt.plot(bins[:-1], histogram_delta)
plt.plot(bins[:-1][outlier_idx], histogram_delta[outlier_idx], '*')
plt.show()



plt.figure()
_, _, _ = plt.hist(histogram_delta, bins=20)
plt.show()

time_range = express_bins_as_sec_range(bins, outlier_idx)

In [ ]:
plt.figure(figsize=(18, 3))
histogram, bins, _ = plt.hist(chat_time_list, bins=max_min)
plt.show()

plt.figure(figsize=(18, 6))
plt.plot(bins[:-1], histogram)
historgram_avg = np.convolve(histogram, np.ones(10)/10, 'same')
plt.plot(bins[:-1], historgram_avg)
plt.show()

histogram_delta = histogram - historgram_avg

print(np.mean(histogram_delta))

print(np.std(histogram_delta))

outlier_idx = np.argwhere(histogram_delta > np.mean(histogram_delta) + 2.5 * np.std(histogram_delta))
outlier_idx = list(itertools.chain.from_iterable(outlier_idx))

print(outlier_idx)
express_bins_as_time_range(bins, outlier_idx)

plt.figure(figsize=(18, 6))
plt.plot(bins[:-1], histogram_delta)
plt.plot(bins[:-1][outlier_idx], histogram_delta[outlier_idx], '*')
plt.show()


plt.figure()
_, _, _ = plt.hist(histogram_delta, bins=20)
plt.show()

In [ ]:
print(datetime.now().strftime("%H-%M-%S"))

In [ ]:
group_index = 0
search_range = time_range[group_index]
chat_group = [list() for i in range(len(time_range))]

for chat in chat_history:
    time = getattr(chat, 'time')
    
    if time >= search_range[0] and time <= search_range[1]:
        chat_group[group_index].append(chat)
    
    if time >= search_range[0] and time >= search_range[1]:
        group_index += 1
        if group_index >= len(time_range):
            break
        else:
            search_range = time_range[group_index]

# for begin, end in time_range:
    

In [ ]:
print(time_range)

In [ ]:
text_group = list()

for chats in chat_group:
    
    text_list = list()
    
    for chat in chats:
        text = getattr(chat, 'text')
        text_list.append(text)
        
    text_group.append(text_list)

In [ ]:
# https://lovit.github.io/nlp/2018/04/17/word_cloud/

min_count = 5   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

# keywords, rank, graph = wordrank_extractor.extract(text_group[0], beta, max_iter)

# for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:30]:
#     print('%8s:\t%.4f' % (word, r))

In [ ]:
font_path = '/Library/Fonts/NanumBarunGothic.ttf'

wordcloud = WordCloud(
    font_path = font_path,
    width = 800,
    height = 800,
    background_color="white"
)

fig, axes = plt.subplots(int(len(text_group) // 2 + 1), 2, figsize=(10, int(len(text_group) // 2 + 1) * 5))

axes = axes.flatten()

for i, texts in enumerate(text_group):
    
    wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)

    keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

    for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
        print('%8s:\t%.4f' % (word, r))
    print()

    wordcloud = wordcloud.generate_from_frequencies(keywords)

    array = wordcloud.to_array()
    # print(type(array)) # numpy.ndarray
    # print(array.shape) # (800, 800, 3)

    axes[i].imshow(array, interpolation="bilinear")
    
plt.show()

In [ ]:
video_file_name = mp4_files[0]

In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

for i, (start, end) in enumerate(time_range):
    suffix = '_{:02d}.mp4'.format(i)
    clip_file_name = video_file_name.replace('.mp4', suffix)
    pre_offset = 10
    ffmpeg_extract_subclip(video_file_name, start - pre_offset, end, targetname=clip_file_name)

In [ ]:
mp4_files = sorted(glob.glob('../videos/*.mp4'))

In [ ]:
# https://www.learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(mp4_files[0])


In [ ]:
# https://www.learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/

import cv2

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(mp4_files[0])

# Check if camera opened successfully
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

# Read until video is completed
while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    if ret == True:
        cv2.imshow('Frame',frame)
#         time.sleep(0.01)
        
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    else: 
        break

cap.release()

# Closes all the frames
cv2.destroyAllWindows()
